In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn import model_selection, linear_model
from sklearn.impute import KNNImputer
import util

In [ ]:
# Path: data-processing.ipynb
# Read in the data
df = pd.read_csv('data/application_train.csv')
df.head(15)

In [ ]:
# Removes columns we first deemed are useless
util.defaultClean(df)

In [ ]:
# All columns with NaNs
df2 = df.loc[:, df.isnull().any()]
print("numCols: ", len(df2.columns))
df2.columns

In [ ]:
# Calculates percentage of NaNs alongside data type of column
percentNull = {}
for col in df.columns:
    percent = (len(df[df[col].isnull()]))/len(df)
    if percent > 0:
        percentNull[col] = (df.dtypes[col], percent)
percentNull

In [ ]:
# Shows all rows where a certain column has NaNs
df[df['EXT_SOURCE_2'].isnull()]

In [ ]:
# Y'ALL, WE'RE DUMB AS SHIT. Check this out:
col_descriptions = pd.read_csv('data/HomeCredit_columns_description.csv', encoding = "ISO-8859-1")
col_descriptions.head(20)

| Column Name | DType | What it Means | How to Handle |
| --- | --- | --- | --- |
| AMT_ANNUITY | float | Monthly payment for the loan | I would find the mean of AMT_ANNUITY/AMT_CREDIT and then multiply that mean ratio by the AMT_CREDIT for the rows that don't have a NaN AMT_ANNUITY |
| AMT_GOODS_PRICE | float | Price of the Good that people get a loan for | Most times, AMT_CREDIT is either slightly greater than or equal to AMT_GOODS_PRICE, so I would just set NaNs in AMT_GOODS_PRICE to whatever the AMT_CREDIT is for that row|
| NAME_TYPE_SUITE | String | Who was accompanying the person when they were applying for the loan | Either just assume they were unnacompanied, use the mode, or use K-Means |
| OWN_CAR_AGE | float | Age of the car they owned | Fill NaNs w/ 0 or -1, since a person without a car will have a NaN here. Filling with 0 as it could conflate with those who have a brand new car, but -1 could be mistinterpreted as a very very new car or something |
| OCCUPATION_TYPE | String | What the person's occupation is | Not sure, probably should leave to K-Means |
| CNT_FAM_MEMBERS | float | How many family members the person has | K-Means or mode |
| EXT_SOURCE_1 | float | Normalized score from external database | Linear Regression/Use other two scores. There are situations in which only one score is present, and very few where no external scores are present. |
| EXT_SOURCE_2 | float | Normalized score from external database | Linear Regression/Use other two scores. There are situations in which only one score is present, and very few where no external scores are present. |
| EXT_SOURCE_3 | float | Normalized score from external database | Linear Regression/Use other two scores. There are situations in which only one score is present, and very few where no external scores are present. |
| OBS_30_CNT_SOCIAL_CIRCLE | float | Amount of people in person's social circle who were observed with a possible 30 DPD (Days past due) default | NaN means there was no observation, so I would make an "assumption" of 0, but you could also use K-Means I think |
| DEF_30_CNT_SOCIAL_CIRCLE | float | Amount of people in person's social circle who defaulted with a  30 DPD (Days past due) | NaN means there was no observation, so I would make an "assumption" of 0, but you could also use K-Means I think |
| OBS_60_CNT_SOCIAL_CIRCLE | float | Amount of people in person's social circle who were observed with a possible 60 DPD (Days past due) default | NaN means there was no observation, so I would make an "assumption" of 0, but you could also use K-Means I think |
| DEF_60_CNT_SOCIAL_CIRCLE | float | Amount of people in person's social circle who defaulted with a 60 DPD (Days past due) | NaN means there was no observation, so I would make an "assumption" of 0, but you could also use K-Means I think |
| DAYS_LAST_PHONE_CHANGE | float | How many days before applying did the client change their phone | I think there's only one NaN so we can set it to 0 |
| AMT_REQ_CREDIT_BUREAU_HOUR | float | Amount of enquiries the client had with the Credit Bureau one hour before application | If there's no data, we probably should assume 0. We can't really guess at how many enquiries a client had and when they had them |
| AMT_REQ_CREDIT_BUREAU_DAY | float | Amount of enquiries the client had with the Credit Bureau one day before application | If there's no data, we probably should assume 0. We can't really guess at how many enquiries a client had and when they had them |
| AMT_REQ_CREDIT_BUREAU_WEEK | float | Amount of enquiries the client had with the Credit Bureau one week before application | If there's no data, we probably should assume 0. We can't really guess at how many enquiries a client had and when they had them |
| AMT_REQ_CREDIT_BUREAU_MON | float | Amount of enquiries the client had with the Credit Bureau one month before application | If there's no data, we probably should assume 0. We can't really guess at how many enquiries a client had and when they had them |
| AMT_REQ_CREDIT_BUREAU_QRT | float | Amount of enquiries the client had with the Credit Bureau three months before application | If there's no data, we probably should assume 0. We can't really guess at how many enquiries a client had and when they had them |
| AMT_REQ_CREDIT_BUREAU_YEAR | float | Amount of enquiries the client had with the Credit Bureau one year before application | If there's no data, we probably should assume 0. We can't really guess at how many enquiries a client had and when they had them |


## 80/20 train-test data split

In [ ]:
df, test_df = model_selection.train_test_split(df, test_size=0.2, shuffle=True)
print("training size:", len(df))
print("testing size:", len(test_df))

# Data Preprocessing

In [ ]:
# All rows where AMT_ANNUITY is NaN
df_annuity = df[df["AMT_ANNUITY"].notnull()]
# Average of non NaN AMT_ANNUITY
numerator = df_annuity["AMT_ANNUITY"].mean()
# Average of non NaN AMT_CREDIT
denominator = df_annuity["AMT_CREDIT"].mean()
# Average ratio of AMT_ANNUITY / AMT_CREDIT
ratio = numerator / denominator
# Set Nulls in AMT_ANNUITY to ratio multiplied by the relevant AMT_CREDIT index
df["AMT_ANNUITY"] = df["AMT_ANNUITY"].fillna(ratio * df["AMT_CREDIT"])

 # Set Nulls in AMT_GOODS_PRICE to the relevant AMT_CREDIT index
df["AMT_GOODS_PRICE"] = df["AMT_GOODS_PRICE"].fillna(df["AMT_CREDIT"])

# Columns where we fill null with 0
cols = ["OWN_CAR_AGE", "OBS_30_CNT_SOCIAL_CIRCLE", "DEF_30_CNT_SOCIAL_CIRCLE",
        "OBS_60_CNT_SOCIAL_CIRCLE", "DEF_60_CNT_SOCIAL_CIRCLE", "DAYS_LAST_PHONE_CHANGE",
        "AMT_REQ_CREDIT_BUREAU_HOUR", "AMT_REQ_CREDIT_BUREAU_DAY", "AMT_REQ_CREDIT_BUREAU_WEEK",
        "AMT_REQ_CREDIT_BUREAU_MON", "AMT_REQ_CREDIT_BUREAU_QRT", "AMT_REQ_CREDIT_BUREAU_YEAR"]
# Set Nulls in cols to 0
for col in cols:
        df[col] = df[col].fillna(0)

In [ ]:
knn_imputer = KNNImputer(n_neighbors=3)
knn_imputer.fit_transform(df)

In [ ]:
non_null_ext_df =  df.dropna(subset=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'])
# non_null_ext_df = df.fillna(value={'EXT_SOURCE_1': 0.5, 'EXT_SOURCE_2': 0.5, 'EXT_SOURCE_3': 0.5})
print(non_null_ext_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']])
print("# points that have all 3 non-null ext sources:", len(non_null_ext_df))
non_null_ext_sample = non_null_ext_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sample(frac=0.001)
# extsrc_regr = linear_model.LinearRegression().fit()
ext_src_fig = px.scatter_3d(non_null_ext_sample, x='EXT_SOURCE_1', y='EXT_SOURCE_2', z='EXT_SOURCE_3')
ext_src_fig.show()
print("correlation between sources 1 and 2 compared to source2\n",
      non_null_ext_df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].corr()['EXT_SOURCE_2'][:], sep='')